In [1]:
import pandas as pd
import numpy as np
import itertools
import pandas_gbq
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
*
FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
# WHERE A.Year=2024
# AND A.GP="Monaco Grand Prix"
# AND A.DRIVER='HAM'
ORDER BY LapNumber, LapStartTime
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [21]:
track2=track3[(track3["GP"]=='Mexico City Grand Prix')&(track3["Year"]==2024)]

In [22]:
track2.head()
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [23]:
track2['LapTime']= pd.to_timedelta(track2["LapTime"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_27180/4173085553.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
# track2['LapTime'].dt.total_seconds().min()*1.07
track2=track2[track2['LapNumber']!=1.0]
track2.loc[:, "LapTime (s)"] = track2["LapTime"].dt.total_seconds()

In [25]:
quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
quicklaps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s)
74878,0 days 01:10:05.336000,VER,1,0 days 00:01:21.442000,7.0,1.0,NaT,NaT,0 days 00:00:29.158000,0 days 00:00:31.408000,0 days 00:00:20.876000,0 days 01:09:13.052000,0 days 01:09:44.460000,0 days 01:10:05.336000,283.0,277.0,245.0,321.0,True,MEDIUM,7.0,True,Red Bull Racing,0 days 01:08:43.894000,NaT,1,1.0,False,,False,True,2024,Mexico City Grand Prix,81.442
74879,0 days 01:10:06.402000,SAI,55,0 days 00:01:21.830000,7.0,1.0,NaT,NaT,0 days 00:00:29.087000,0 days 00:00:32.010000,0 days 00:00:20.733000,0 days 01:09:13.624000,0 days 01:09:45.634000,0 days 01:10:06.367000,276.0,281.0,248.0,329.0,True,MEDIUM,7.0,True,Ferrari,0 days 01:08:44.572000,NaT,1,2.0,False,,False,True,2024,Mexico City Grand Prix,81.830
74880,0 days 01:10:07.182000,NOR,4,0 days 00:01:21.885000,7.0,1.0,NaT,NaT,0 days 00:00:29.172000,0 days 00:00:31.879000,0 days 00:00:20.834000,0 days 01:09:14.427000,0 days 01:09:46.306000,0 days 01:10:07.140000,290.0,277.0,247.0,324.0,True,MEDIUM,7.0,True,McLaren,0 days 01:08:45.297000,NaT,1,3.0,False,,False,True,2024,Mexico City Grand Prix,81.885
74881,0 days 01:10:08.063000,LEC,16,0 days 00:01:22.154000,7.0,1.0,NaT,NaT,0 days 00:00:28.820000,0 days 00:00:32.362000,0 days 00:00:20.972000,0 days 01:09:14.712000,0 days 01:09:47.074000,0 days 01:10:08.046000,290.0,283.0,247.0,324.0,True,MEDIUM,7.0,True,Ferrari,0 days 01:08:45.909000,NaT,1,4.0,False,,False,True,2024,Mexico City Grand Prix,82.154
74882,0 days 01:10:09.127000,HAM,44,0 days 00:01:22.609000,7.0,1.0,NaT,NaT,0 days 00:00:29.160000,0 days 00:00:32.251000,0 days 00:00:21.198000,0 days 01:09:15.645000,0 days 01:09:47.896000,0 days 01:10:09.094000,289.0,276.0,246.0,329.0,True,MEDIUM,7.0,True,Mercedes,0 days 01:08:46.518000,NaT,1,5.0,False,,False,True,2024,Mexico City Grand Prix,82.609


In [26]:
transformed_laps = quicklaps.copy()
transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

# order the team from the fastest (lowest median lap time) to slower
team_order = (
    transformed_laps[["Team", "LapTime (s)"]].groupby("Team").median()["LapTime (s)"].sort_values().index
)
print(team_order)

Index(['McLaren', 'Ferrari', 'Mercedes', 'Red Bull Racing', 'Haas F1 Team',
       'Alpine', 'Aston Martin', 'Williams', 'RB', 'Kick Sauber'],
      dtype='object', name='Team')


In [27]:
transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('median')
transformed_laps.tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s),median
585995,0 days 02:37:42.418000,HAM,44,0 days 00:01:20.942000,71.0,2.0,NaT,NaT,0 days 00:00:28.717000,0 days 00:00:31.351000,0 days 00:00:20.874000,0 days 02:36:50.160000,0 days 02:37:21.511000,0 days 02:37:42.385000,NaN,284.0,250.0,330.0,False,HARD,43.0,True,Mercedes,0 days 02:36:21.476000,NaT,1,4.0,False,,False,True,2024,Mexico City Grand Prix,80.942,81.436
585997,0 days 02:37:46.174000,RUS,63,0 days 00:01:21.359000,71.0,2.0,NaT,NaT,0 days 00:00:28.862000,0 days 00:00:31.507000,0 days 00:00:20.990000,0 days 02:36:53.663000,0 days 02:37:25.170000,0 days 02:37:46.160000,299.0,284.0,250.0,NaN,False,HARD,40.0,True,Mercedes,0 days 02:36:24.815000,NaT,1,5.0,False,,False,True,2024,Mexico City Grand Prix,81.359,81.436
586005,0 days 02:37:57.196000,VER,1,0 days 00:01:21.570000,71.0,2.0,NaT,NaT,0 days 00:00:28.933000,0 days 00:00:31.610000,0 days 00:00:21.027000,0 days 02:37:04.559000,0 days 02:37:36.169000,0 days 02:37:57.196000,288.0,284.0,250.0,328.0,False,HARD,45.0,True,Red Bull Racing,0 days 02:36:35.626000,NaT,1,6.0,False,,False,True,2024,Mexico City Grand Prix,81.570,81.712
586008,0 days 02:38:01.280000,MAG,20,0 days 00:01:21.905000,71.0,2.0,NaT,NaT,0 days 00:00:28.589000,0 days 00:00:32.066000,0 days 00:00:21.250000,0 days 02:37:07.939000,0 days 02:37:40.005000,0 days 02:38:01.255000,297.0,284.0,252.0,334.0,False,HARD,41.0,True,Haas F1 Team,0 days 02:36:39.375000,NaT,1,7.0,False,,False,True,2024,Mexico City Grand Prix,81.905,81.832
586010,0 days 02:38:02.566000,PIA,81,0 days 00:01:21.531000,71.0,2.0,NaT,NaT,0 days 00:00:28.557000,0 days 00:00:31.332000,0 days 00:00:21.642000,0 days 02:37:09.558000,0 days 02:37:40.890000,0 days 02:38:02.532000,297.0,286.0,247.0,335.0,False,HARD,32.0,True,McLaren,0 days 02:36:41.035000,NaT,1,8.0,False,,False,True,2024,Mexico City Grand Prix,81.531,80.881


In [28]:
fig_pace=px.box(
    transformed_laps.sort_values(by=["median","LapNumber"]),
    x="Team",
    y="LapTime (s)",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace Plot for the {} {}</b>".format(year,gp),
    # palette=team_palette,
    # whiskerprops=dict(color="white"),
    # boxprops=dict(edgecolor="white"),
    # medianprops=dict(color="grey"),
    # capprops=dict(color="white"),
    height=700, width=1200,

color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)
fig_pace.update_traces(opacity=1)
fig_pace.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    margin=dict(l=60, r=5, t=50, b=60),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pace.show()

In [29]:
fig_pace.write_html("/Users/rdesh723/statpulse-html/plots/Race Team Pace/Team Pace Plots/{}/{} {} Team Pace Plot.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [30]:
#Get MINIMUM OF THE MEDIAN LAP TIMES

fastest_lap = transformed_laps["median"].min()
transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
transformed_laps_relative_median=transformed_laps[["Team","Delta","Delta_Percent"]].drop_duplicates()
transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
transformed_laps_relative_median

,Team,Delta,Delta_Percent
74880,McLaren,0.0000,100.0%
74879,Ferrari,0.0910,100.113%
74882,Mercedes,0.5550,100.686%
74878,Red Bull Racing,0.8310,101.027%
74884,Haas F1 Team,0.9510,101.176%
74886,Alpine,1.8310,102.264%
74888,Aston Martin,1.8410,102.276%
74891,Williams,1.9275,102.383%
74887,RB,1.9340,102.391%
74892,Kick Sauber,2.2310,102.758%


In [31]:
for index, row in transformed_laps_relative_median.iterrows():
  if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
    transformed_laps_relative_median.loc[index, 'Delta']=0.001
transformed_laps_relative_median

,Team,Delta,Delta_Percent
74880,McLaren,0.0010,100.0%
74879,Ferrari,0.0910,100.113%
74882,Mercedes,0.5550,100.686%
74878,Red Bull Racing,0.8310,101.027%
74884,Haas F1 Team,0.9510,101.176%
74886,Alpine,1.8310,102.264%
74888,Aston Martin,1.8410,102.276%
74891,Williams,1.9275,102.383%
74887,RB,1.9340,102.391%
74892,Kick Sauber,2.2310,102.758%


In [32]:
fig_median=px.bar(
    transformed_laps_relative_median,
    x="Team",
    y="Delta",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace (Median Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent",
    height=700, width=1200,
color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)

fig_median.update_traces(textposition='outside')

fig_median.update_traces(opacity=0.85)

fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

# fig['layout']['yaxis']['autorange'] = "reversed"

fig_median.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_median.update_xaxes(title_text='')      # Remove axis titles
fig_median.update_yaxes(title_text='')

fig_median.update_xaxes(
        title_standoff = 75
)

fig_median.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_median.show()

In [33]:
fig_median.write_html("/Users/rdesh723/statpulse-html/plots/Race Team Pace/Median Lap Times/{}/{} {} Team Median Plot.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [34]:
#Get MINIMUM OF ALL LAP TIMES
transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('min')

fastest_lap_all = transformed_laps["min_all"].min()
transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
transformed_laps_relative_min_all=transformed_laps[["Team","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
transformed_laps_relative_min_all

,Team,Delta_Min_All,Delta_Percent_All
74879,Ferrari,0.000,100.0%
74878,Red Bull Racing,0.873,101.114%
74887,RB,1.166,101.488%
74880,McLaren,1.355,101.73%
74891,Williams,1.754,102.239%
74882,Mercedes,2.035,102.598%
74884,Haas F1 Team,2.301,102.937%
74886,Alpine,2.323,102.965%
74892,Kick Sauber,2.749,103.509%
74888,Aston Martin,2.975,103.798%


In [35]:
for index, row in transformed_laps_relative_min_all.iterrows():
  if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
    transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
transformed_laps_relative_min_all

,Team,Delta_Min_All,Delta_Percent_All
74879,Ferrari,0.001,100.0%
74878,Red Bull Racing,0.873,101.114%
74887,RB,1.166,101.488%
74880,McLaren,1.355,101.73%
74891,Williams,1.754,102.239%
74882,Mercedes,2.035,102.598%
74884,Haas F1 Team,2.301,102.937%
74886,Alpine,2.323,102.965%
74892,Kick Sauber,2.749,103.509%
74888,Aston Martin,2.975,103.798%


In [36]:
fig_min=px.bar(
    transformed_laps_relative_min_all,
    x="Team",
    y="Delta_Min_All",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent_All",
    height=700, width=1200,
color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)

fig_min.update_traces(textposition='outside')

fig_min.update_traces(opacity=0.85)

fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

# fig['layout']['yaxis']['autorange'] = "reversed"

fig_min.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_min.update_xaxes(title_text='')      # Remove axis titles
fig_min.update_yaxes(title_text='')

fig_min.update_xaxes(
        title_standoff = 75
)

fig_min.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_min.update_yaxes(ticksuffix = "  ")
fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_min.show()

In [37]:
fig_min.write_html("/Users/rdesh723/statpulse-html/plots/Race Team Pace/Fastest Lap/{}/{} {} Team Fastest Lap Plot.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [21]:
#Multi GP Plot

In [32]:
#2024
race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
# race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2024
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    track2['LapTime']= pd.to_timedelta(track2["LapTime"])
    track2=track2[track2['LapNumber']!=1.0]
    quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
    transformed_laps = quicklaps.copy()
    transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

    # order the team from the fastest (lowest median lap time) tp slower
    team_order = (
        transformed_laps[["Team", "LapTime (s)"]].groupby("Team").median()["LapTime (s)"].sort_values().index
    )
    print(team_order)
    transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('median')
    transformed_laps.tail()
    fig_pace=px.box(
        transformed_laps.sort_values(by=["median","LapNumber"]),
        x="Team",
        y="LapTime (s)",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace Plot for the {} {}</b>".format(year_multi,i),
        height=700, width=1200,

    color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            }
    )
    fig_pace.update_traces(opacity=1)
    fig_pace.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow",
        margin=dict(l=50, r=5, t=35, b=50)
    )
    fig_pace.write_html("/Users/rdesh723/statpulse-html/plots/Race Team Pace/Team Pace Plots/{}/{} {} Team Pace Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')
    #Get MINIMUM OF THE MEDIAN LAP TIMES

    fastest_lap = transformed_laps["median"].min()
    transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
    transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
    transformed_laps_relative_median=transformed_laps[["Team","Delta","Delta_Percent"]].drop_duplicates()
    transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
    transformed_laps_relative_median
    for index, row in transformed_laps_relative_median.iterrows():
        if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
         transformed_laps_relative_median.loc[index, 'Delta']=0.001
    transformed_laps_relative_median
    fig_median=px.bar(
        transformed_laps_relative_median,
        x="Team",
        y="Delta",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace (Median Lap Times) for the {} {}</b>".format(year_multi,i),
        text="Delta_Percent",
        height=700, 
        width=1200,
        color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            }
    )

    fig_median.update_traces(textposition='outside')

    fig_median.update_traces(opacity=0.85)

    fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

    fig_median.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_median.update_xaxes(title_text='')      # Remove axis titles
    fig_median.update_yaxes(title_text='')

    fig_median.update_xaxes(
            title_standoff = 75
    )

    fig_median.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_median.write_html("/Users/rdesh723/statpulse-html/plots/Race Team Pace/Median Lap Times/{}/{} {} Team Median Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

  #Get MINIMUM OF ALL LAP TIMES
    transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('min')
    fastest_lap_all = transformed_laps["min_all"].min()
    transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
    transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
    transformed_laps_relative_min_all=transformed_laps[["Team","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
    transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
    transformed_laps_relative_min_all
    for index, row in transformed_laps_relative_min_all.iterrows():
        if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
            transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
    fig_min=px.bar(
        transformed_laps_relative_min_all,
        x="Team",
        y="Delta_Min_All",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
        text="Delta_Percent_All",
        height=700, width=1200,
    color_discrete_map={
                     "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                    }
    )

    fig_min.update_traces(textposition='outside')

    fig_min.update_traces(opacity=0.85)

    fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

    # fig['layout']['yaxis']['autorange'] = "reversed"

    fig_min.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_min.update_xaxes(title_text='')      # Remove axis titles
    fig_min.update_yaxes(title_text='')

    fig_min.update_xaxes(
            title_standoff = 75
    )

    fig_min.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_min.update_yaxes(ticksuffix = "  ")
    fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_min.write_html("/Users/rdesh723/statpulse-html/plots/Race Team Pace/Fastest Lap/{}/{} {} Team Fastest Lap Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['Red Bull Racing', 'Ferrari', 'McLaren', 'Mercedes', 'Haas F1 Team',
       'Aston Martin', 'Kick Sauber', 'Alpine', 'RB', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'Ferrari', 'McLaren', 'Mercedes', 'Aston Martin',
       'Haas F1 Team', 'Kick Sauber', 'Alpine', 'Williams', 'RB'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['Ferrari', 'McLaren', 'Red Bull Racing', 'Mercedes', 'Aston Martin',
       'Haas F1 Team', 'Alpine', 'RB', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'McLaren', 'Mercedes', 'Ferrari', 'Aston Martin',
       'RB', 'Williams', 'Kick Sauber', 'Haas F1 Team', 'Alpine'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

...

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

Index(['Red Bull Racing', 'Mercedes', 'Williams', 'Ferrari', 'McLaren', 'RB',
       'Alpine', 'Kick Sauber', 'Aston Martin'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'Mercedes', 'McLaren', 'Aston Martin',
       'Haas F1 Team', 'RB', 'Alpine', 'Kick Sauber', 'Ferrari', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'McLaren', 'Ferrari', 'Mercedes', 'Alpine',
       'Haas F1 Team', 'RB', 'Aston Martin', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['McLaren', 'Ferrari', 'Mercedes', 'Red Bull Racing', 'Haas F1 Team',
       'Alpine', 'RB', 'Aston Martin', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')
Index(['McLaren', 'Mercedes', 'Red Bull Racing', 'Ferrari', 'Aston Martin',
       'Haas F1 Team', 'Williams', 'RB', 'Kick Sauber', 'Alpine'],
      dtype='object', name='Team')
Index(['Mercedes', 'Red Bull Racing', 'McLaren', 'Ferrari', 'Aston Martin',
       'RB', 'Haas F1 Team', 'Williams', 'Kick Sauber', 'Alpine'],
      dtype='object', name='Team')
Index(['Mercedes', 'McLaren', 'Ferrari', 'Red Bull Racing', 'Williams', 'RB',
       'Alpine', 'Aston Martin', 'Kick Sauber', 'Haas F1 Team'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [253]:
# #Get AVERAGE OF ALL LAP TIMES
# transformed_laps['avg_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform(np.mean)

# fastest_lap_avg = transformed_laps["avg_all"].min()
# transformed_laps['Delta_Avg_All']= transformed_laps["avg_all"] - (fastest_lap_avg)
# transformed_laps['Delta_Percent_Avg']= (transformed_laps["avg_all"]/(fastest_lap_avg)-1)*100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].round(3)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].astype(str)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+"%"
# transformed_laps_relative_avg_all=transformed_laps[["Team","Delta_Avg_All","Delta_Percent_Avg"]].drop_duplicates()
# transformed_laps_relative_avg_all=transformed_laps_relative_avg_all.sort_values("Delta_Avg_All")
# transformed_laps_relative_avg_all

In [254]:
# for index, row in transformed_laps_relative_avg_all.iterrows():
#   if transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']==0.0:
#     transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']=0.001
# transformed_laps_relative_avg_all

In [255]:
# fig=px.bar(
#     transformed_laps_relative_avg_all,
#     x="Team",
#     y="Delta_Avg_All",
#     color='Team',
#     template="xgridoff",
#     title="<b>Team Pace (Average Lap Times) for the {} {}</b>".format(year,gp),
#     text="Delta_Percent_Avg",
#     height=700, width=1200,
# color_discrete_map={
#                  "Alpine": "#0093cc",
#                  "Aston Martin": "#229971",
#                  "Ferrari": "#E80020",
#                  "Haas F1 Team": "#B6BABD",
#                  "Kick Sauber": "#52e252",
#                  "McLaren": "#FF8000",
#                  "Mercedes": "#27F4D2",
#                  "RB": "#6692FF",
#                  "Red Bull Racing": "#3671C6",
#                  "Williams": "#64C4FF" ,
#                  "Alfa Romeo":"#C92D4B",
#                  "AlphaTauri":"#5E8FAA",
#                  "Racing Point":"#F596C8",
#                  "Renault":"#FFF500",
#                  "Toro Rosso":"#469bff",
#                  "Force India":"#F596C8",
#                  "Sauber":"#9B0000"
#                  }
# )

# fig.update_traces(textposition='outside')

# fig.update_traces(opacity=0.85)

# fig.update_traces( marker_line_color='white',marker_line_width=1.5)

# fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_yaxes(range=[2, 0])

# # fig['layout']['yaxis']['autorange'] = "reversed"

# fig.update_layout(
#     xaxis={'side': 'top'},
#     yaxis={'side': 'left'}
# )

# fig.update_xaxes(title_text='')      # Remove axis titles
# fig.update_yaxes(title_text='')

# fig.update_xaxes(
#         title_standoff = 75
# )

# fig.update_layout(
#     title_x=0.5,
#     margin=dict(l=60, r=5, t=110, b=30),
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=14,
#         color="Black"
#     ),
#     title_font_family="PT Sans Narrow"
# )
# fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

# fig.show()

In [256]:
# fig.write_html("{} {} Team Avg Lap Plot.html".format(year,gp))